### Train the transformer!

##### Three important modules
1. `MultiHeadSelfAttention`: A self-attention implementation
2. `Block`: A transformer block which is repeated n_layer times in a GPT model
3. `GPT`: The full GPT model itself, including initial embeddings, the GPT blocks, and the token decoding logic.

The `GPT` module uses the `Block` module, which in turn uses the `MultiHeadSelfAttention` module.
```
    ┌────────────────────────┐
    │             GPT               │
    └────────────────────────┘
                   ▲
    ┌───────────┴────────────┐
    │            Block              │
    └────────────────────────┘
                   ▲
    ┌───────────┴────────────┐
    │    MultiHeadSelfAttention     │
    └────────────────────────┘
```

In [1]:
from common import GPTConfig, MultiHeadSelfAttention

/home/sriramramesh/miniconda3/envs/drl_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Let's use a placeholder config to show how the attention layer works
config = GPTConfig(
    vocab_size=10,
    n_layer=3,
    n_embd=12,
    n_head=4,
    block_size=5,
)

In [5]:
attention = MultiHeadSelfAttention(config)

In [6]:
print(attention)

MultiHeadSelfAttention(
  (key): Linear(in_features=12, out_features=12, bias=True)
  (query): Linear(in_features=12, out_features=12, bias=True)
  (value): Linear(in_features=12, out_features=12, bias=True)
  (attn_drop): Dropout(p=0.1, inplace=False)
  (resid_drop): Dropout(p=0.1, inplace=False)
  (proj): Linear(in_features=12, out_features=12, bias=True)
)


In [7]:
import torch.nn as nn
from common import MultiHeadSelfAttention

In [8]:
class Block(nn.Module):
    """ An unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)

        self.attn = MultiHeadSelfAttention(config)
        # MLP is a feed-forward neural network
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(0.1),
        )
    
    def forward(self, x):
        x = x + self.attn(self.ln1(x)) + self.mlp(self.ln2(x))
        return x

In [9]:
block = Block(config)
block

Block(
  (ln1): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
  (ln2): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
  (attn): MultiHeadSelfAttention(
    (key): Linear(in_features=12, out_features=12, bias=True)
    (query): Linear(in_features=12, out_features=12, bias=True)
    (value): Linear(in_features=12, out_features=12, bias=True)
    (attn_drop): Dropout(p=0.1, inplace=False)
    (resid_drop): Dropout(p=0.1, inplace=False)
    (proj): Linear(in_features=12, out_features=12, bias=True)
  )
  (mlp): Sequential(
    (0): Linear(in_features=12, out_features=48, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=48, out_features=12, bias=True)
    (3): Dropout(p=0.1, inplace=False)
  )
)

In [10]:
assert isinstance(block.attn, MultiHeadSelfAttention)

In [11]:
import torch
import torch.nn.functional as F

In [12]:
class GPT(nn.Module):
    """ The full GPT language model, with a context size of block_size """

    def __init__(self, config):
        super().__init__()

        # Input embedding stem
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.embd_pdrop)

        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        
        # Decoder head
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size
        self.apply(self._init_weights)
        
        print("Number of parameters: {}".format(sum(p.numel() for p in self.parameters())))

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def get_block_size(self):
        return self.block_size

    def forward(self, idx, targets=None):
        b, t = idx.size()
        assert t <= self.block_size, "Cannot forward, model block size is exhausted."

        # Create token embeddings and add positional embeddings
        # Each index maps to a (learnable) vector
        token_embeddings = self.tok_emb(idx)
        # Each position maps to a (learnable) vector
        position_embeddings = self.pos_emb[:, :t, :]
        
        x = self.drop(token_embeddings + position_embeddings)
        x = self.blocks(x)

        # Decode the output of the transformer blocks
        x = self.ln_f(x)
        logits = self.head(x)

        # If we are given some desired targets also calculate the loss, e.g. during training
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            loss = None

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, stop_tokens=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        
        for _ in range(max_new_tokens):
            # If the sequence context is growing too long we must crop it at block_size
            idx_cond = (idx if idx.size(1) <= self.block_size else idx[:, -self.block_size :])

            # Forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)

            # Pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature

            # Optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float("Inf")

            # Apply softmax to convert logits to (normalized) probabilities
            # using F.softmax.
            probs = F.softmax(logits, dim=-1)

            # Sample from the distribution (if top_k=1 this is equivalent to greedy sampling)
            # using torch.multinomial. Only need to sample a single token.
            idx_next = torch.multinomial(probs, num_samples=1)

            # Append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

            # Stop prediction if we produced a stop token
            if stop_tokens is not None and idx_next.item() in stop_tokens:
                return idx

        return idx

In [13]:
model = GPT(config)

Number of parameters: 5976


In [14]:
input_seq = torch.tensor([[1, 2, 3]])

In [15]:
output_seq = model.generate(input_seq, max_new_tokens=30)

In [16]:
output_seq

tensor([[1, 2, 3, 7, 9, 2, 0, 7, 6, 9, 5, 8, 6, 0, 8, 6, 0, 6, 1, 1, 2, 8, 3, 7,
         5, 2, 1, 4, 6, 2, 8, 9, 5]])

In [17]:
# Check the generated sequence shape
assert output_seq.shape == (1, 33)

In [18]:
output_seq.shape

torch.Size([1, 33])

In [19]:
print("input sequence:", input_seq.tolist()[0])
print("output sequence:", output_seq.tolist()[0])

print("Success!")

input sequence: [1, 2, 3]
output sequence: [1, 2, 3, 7, 9, 2, 0, 7, 6, 9, 5, 8, 6, 0, 8, 6, 0, 6, 1, 1, 2, 8, 3, 7, 5, 2, 1, 4, 6, 2, 8, 9, 5]
Success!


#### Load a dataset

In [20]:
import numpy as np
from common import AdditionDataset, CharacterTokenizer

In [21]:
BLOCK_SIZE = 32  # context length

In [23]:
tokenizer = CharacterTokenizer(
    characters=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", " ", "="],
    model_max_length=BLOCK_SIZE,
)

In [24]:
tokenizer

CharacterTokenizer(name_or_path='', vocab_size=20, model_max_length=32, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[BOS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("[BOS]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
	4: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	6: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [25]:
dataset = AdditionDataset(
    tokenizer=tokenizer,
    block_size=BLOCK_SIZE,
    numbers=list(range(0, 1000, 2)),
    include_intermediate_steps=True,
)

In [26]:
tokenizer.vocab_size

20

In [28]:
c = 5129836
c_parts = [(int(s) * 10**i) for i, s in enumerate(str(c)[::-1])]
c_parts

[6, 30, 800, 9000, 20000, 100000, 5000000]

In [32]:
for ix in [11177, 22222]:
    x, y = dataset[ix]
    print(f"=== Example {ix} ===")
    
    np.set_printoptions(linewidth=999)
    print(f"x = {x.numpy()}")
    print(f"y = {y.numpy()}")
    
    # Show lengths
    print(f"x length = {len(x)}")
    print(f"y length = {len(y)}")
    
    # Print x decoded
    x = tokenizer.decode(x, skip_special_tokens=True)
    print(f"x decoded = {x}")
    
    # Print y decoded, replacing the -1 token with _
    num_unknowns = y.tolist().count(-1)
    y = tokenizer.decode(y[num_unknowns:], skip_special_tokens=True)
    print(f"y decoded = {'_'*(num_unknowns-1)}{y}")

=== Example 11177 ===
x = [ 0 11 11 17 10 12 11 19 15 17 16  7 17 10  7  7 19 10 16 15  1  4  4  4  4  4  4  4  4  4  4  4]
y = [-1 -1 -1 -1 -1 -1 -1 15 17 16  7 17 10  7  7 19 10 16 15  1  4  4  4  4  4  4  4  4  4  4  4  4]
x length = 32
y length = 32
x decoded = 44+354=8+90+300=398
y decoded = ______8+90+300=398
=== Example 22222 ===
x = [ 0 15 15 17 11 11 11 19  9 17 10  7 17 12  7  7 19 12 10  9  1  4  4  4  4  4  4  4  4  4  4  4]
y = [-1 -1 -1 -1 -1 -1 -1  9 17 10  7 17 12  7  7 19 12 10  9  1  4  4  4  4  4  4  4  4  4  4  4  4]
x length = 32
y length = 32
x decoded = 88+444=2+30+500=532
y decoded = ______2+30+500=532


In [39]:
# Train the model
from common import GPTConfig, Trainer, TrainerConfig

In [40]:
# Instantiate a mini-GPT type model
model_config = GPTConfig(
    vocab_size=tokenizer.vocab_size,
    block_size=BLOCK_SIZE,
    n_layer=3,
    n_head=3,
    n_embd=48,
)

In [41]:
model = GPT(model_config)

Number of parameters: 88368


In [42]:
# Create a trainer 
train_config = TrainerConfig(max_epochs=1, batch_size=1000, learning_rate=4e-3)

In [43]:
trainer = Trainer(model, dataset, train_config)

In [44]:
print(f"Using device: {trainer.device}")

Using device: 0


In [ ]:
from common import show_examples

# Sample output before training the model
show_examples(model, dataset, tokenizer, trainer.device, top_k=1, temperature=1.0)

❌ input->output: [CLS]550+84= 👉 [CLS] expected: 634, got: None
❌ input->output: [CLS]558+504= 👉 92[MASK]=92[MASK]=92[MASK]82[MASK][SEP] expected: 1062, got: 92
❌ input->output: [CLS]278+88= 👉 [PAD] [PAD][PAD][BOS]5[MASK]=9+ [PAD][BOS]5[MASK][SEP] expected: 366, got: 9
❌ input->output: [CLS]422+740= 👉 [PAD]9[CLS] expected: 1162, got: None
❌ input->output: [CLS]760+660= 👉 92666[PAD]7[MASK]=904[RESERVED][RESERVED][RESERVED][BOS]92[MASK]=92666[PAD]7=92[MASK]= expected: 1420, got: 92
❌ input->output: [CLS]256+630= 👉 9=9+666=[RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED][RESERVED]7[RESERVED][RESERVED][RESERVED][RESERVED][RESERVED] expected: 886, got: 9
❌ input->output: [CLS]120+672= 👉 9+26[CLS] expected: 792, got: None
❌ input->output: [CLS]94+418= 👉 9+ [UNK]2[MASK]=9+[MASK]=1[BOS]5[MASK]=92+  9269[CLS] expected: 512, got: 92
❌ input->output: [CLS]464+796= 👉 9+86[PAD][B

In [46]:
for _ in range(6):
    trainer.train()  # train just one epoch each time
    show_examples(model, dataset, tokenizer, trainer.device, top_k=3, temperature=1.0)

epoch 1 iter 249: train loss 0.25906. lr 4.000000e-03: 100%|██████████| 250/250 [00:25<00:00,  9.91it/s]


❌ input->output: [CLS]550+84= 👉 4+40+500=544[SEP] expected: 634, got: 544
❌ input->output: [CLS]558+504= 👉 4+50+500+1000=1554[SEP] expected: 1062, got: 1554
❌ input->output: [CLS]278+88= 👉 0+90+0+1000=1090[SEP] expected: 366, got: 1090
❌ input->output: [CLS]422+740= 👉 4+80+600=682[SEP] expected: 1162, got: 682
❌ input->output: [CLS]760+660= 👉 0+20+0+1000=1020[SEP] expected: 1420, got: 1020
❌ input->output: [CLS]256+630= 👉 6+40+800=846[SEP] expected: 886, got: 846
❌ input->output: [CLS]120+672= 👉 2+10+300+1000=1312[SEP] expected: 792, got: 1312
❌ input->output: [CLS]94+418= 👉 0+30+400=4[SEP] expected: 512, got: 4
❌ input->output: [CLS]464+796= 👉 8+0+800=808[SEP] expected: 1260, got: 808
❌ input->output: [CLS]624+106= 👉 0+10+100+1000=1110[SEP] expected: 730, got: 1110

Correct: 0 out of 10: 0.0%


epoch 1 iter 249: train loss 0.06866. lr 4.000000e-03: 100%|██████████| 250/250 [00:25<00:00,  9.88it/s]


✅ input->output: [CLS]550+84= 👉 4+30+600=634[SEP]
✅ input->output: [CLS]558+504= 👉 2+60+0+1000=1062[SEP]
❌ input->output: [CLS]278+88= 👉 6+50+300=356[SEP] expected: 366, got: 356
❌ input->output: [CLS]422+740= 👉 2+60+0+1000=1062[SEP] expected: 1162, got: 1062
❌ input->output: [CLS]760+660= 👉 0+30+400+1000=1430[SEP] expected: 1420, got: 1430
✅ input->output: [CLS]256+630= 👉 6+80+800=886[SEP]
✅ input->output: [CLS]120+672= 👉 2+90+700=792[SEP]
❌ input->output: [CLS]94+418= 👉 2+0+300=302[SEP] expected: 512, got: 302
✅ input->output: [CLS]464+796= 👉 0+60+200+1000=1260[SEP]
❌ input->output: [CLS]624+106= 👉 0+20+700=720[SEP] expected: 730, got: 720

Correct: 5 out of 10: 50.0%


epoch 1 iter 249: train loss 0.03969. lr 4.000000e-03: 100%|██████████| 250/250 [00:25<00:00,  9.96it/s]


...

✅ input->output: [CLS]550+84= 👉 4+30+600=634[SEP]
✅ input->output: [CLS]558+504= 👉 2+60+0+1000=1062[SEP]
❌ input->output: [CLS]278+88= 👉 6+40+300=346[SEP] expected: 366, got: 346
✅ input->output: [CLS]422+740= 👉 2+60+100+1000=1162[SEP]
❌ input->output: [CLS]760+660= 👉 0+40+400+1000=1440[SEP] expected: 1420, got: 1440
✅ input->output: [CLS]256+630= 👉 6+80+800=886[SEP]
✅ input->output: [CLS]120+672= 👉 2+90+700=792[SEP]
✅ input->output: [CLS]94+418= 👉 2+10+500=512[SEP]
✅ input->output: [CLS]464+796= 👉 0+60+200+1000=1260[SEP]
✅ input->output: [CLS]624+106= 👉 0+30+700=730[SEP]

Correct: 8 out of 10: 80.0%


epoch 1 iter 249: train loss 0.01323. lr 4.000000e-03: 100%|██████████| 250/250 [00:25<00:00,  9.82it/s]


✅ input->output: [CLS]550+84= 👉 4+30+600=634[SEP]
✅ input->output: [CLS]558+504= 👉 2+60+0+1000=1062[SEP]
✅ input->output: [CLS]278+88= 👉 6+60+300=366[SEP]
✅ input->output: [CLS]422+740= 👉 2+60+100+1000=1162[SEP]
❌ input->output: [CLS]760+660= 👉 6+20+400+1000=1426[SEP] expected: 1420, got: 1426
✅ input->output: [CLS]256+630= 👉 6+80+800=886[SEP]
✅ input->output: [CLS]120+672= 👉 2+90+700=792[SEP]
❌ input->output: [CLS]94+418= 👉 2+10+400=412[SEP] expected: 512, got: 412
✅ input->output: [CLS]464+796= 👉 0+60+200+1000=1260[SEP]
✅ input->output: [CLS]624+106= 👉 0+30+700=730[SEP]

Correct: 8 out of 10: 80.0%


epoch 1 iter 249: train loss 0.01004. lr 4.000000e-03: 100%|██████████| 250/250 [00:25<00:00,  9.82it/s]


✅ input->output: [CLS]550+84= 👉 4+30+600=634[SEP]
✅ input->output: [CLS]558+504= 👉 2+60+0+1000=1062[SEP]
✅ input->output: [CLS]278+88= 👉 6+60+300=366[SEP]
✅ input->output: [CLS]422+740= 👉 2+60+100+1000=1162[SEP]
✅ input->output: [CLS]760+660= 👉 0+20+400+1000=1420[SEP]
✅ input->output: [CLS]256+630= 👉 6+80+800=886[SEP]
✅ input->output: [CLS]120+672= 👉 2+90+700=792[SEP]
❌ input->output: [CLS]94+418= 👉 2+90+500=592[SEP] expected: 512, got: 592
✅ input->output: [CLS]464+796= 👉 0+60+200+1000=1260[SEP]
✅ input->output: [CLS]624+106= 👉 0+30+700=730[SEP]

Correct: 9 out of 10: 90.0%


In [47]:
# Let's evaluate on 30 examples. No modifications are needed.

show_examples(
    model, dataset, tokenizer, trainer.device, top_k=1, temperature=1.0, max_num=30
)

✅ input->output: [CLS]550+84= 👉 4+30+600=634[SEP]
✅ input->output: [CLS]558+504= 👉 2+60+0+1000=1062[SEP]
✅ input->output: [CLS]278+88= 👉 6+60+300=366[SEP]
✅ input->output: [CLS]422+740= 👉 2+60+100+1000=1162[SEP]
✅ input->output: [CLS]760+660= 👉 0+20+400+1000=1420[SEP]
✅ input->output: [CLS]256+630= 👉 6+80+800=886[SEP]
✅ input->output: [CLS]120+672= 👉 2+90+700=792[SEP]
✅ input->output: [CLS]94+418= 👉 2+10+500=512[SEP]
✅ input->output: [CLS]464+796= 👉 0+60+200+1000=1260[SEP]
✅ input->output: [CLS]624+106= 👉 0+30+700=730[SEP]
❌ input->output: [CLS]72+376= 👉 8+0+400=408[SEP] expected: 448, got: 408
✅ input->output: [CLS]630+824= 👉 4+50+400+1000=1454[SEP]
✅ input->output: [CLS]916+372= 👉 8+80+200+1000=1288[SEP]
✅ input->output: [CLS]246+74= 👉 0+20+300=320[SEP]
✅ input->output: [CLS]626+158= 👉 4+80+700=784[SEP]
✅ input->output: [CLS]414+164= 👉 8+70+500=578[SEP]
✅ input->output: [CLS]738+126= 👉 4+60+800=864[SEP]
✅ input->output: [CLS]600+60= 👉 0+60+600=660[SEP]
✅ input->output: [CLS]470+110= 